# Q1.2: SQL

In [18]:
from pyspark.sql import SparkSession
# import pandas as pd

In [19]:
# pd.read_excel(open("data/01_BI_SQL_Dataset.xlsx", "rb"), sheet_name="order_item").to_csv("data/01/order_item.csv")
# pd.read_excel(open("data/01_BI_SQL_Dataset.xlsx", "rb"), sheet_name="first_purchase").to_csv("data/01/first_purchase.csv")
# pd.read_excel(open("data/01_BI_SQL_Dataset.xlsx", "rb"), sheet_name="shop_level_target").to_csv("data/01/shop_level_target.csv")
# pd.read_excel(open("data/01_BI_SQL_Dataset.xlsx", "rb"), sheet_name="kam_level_target").to_csv("data/01/kam_level_target.csv")
# pd.read_excel(open("data/01_BI_SQL_Dataset.xlsx", "rb"), sheet_name="weightage").to_csv("data/01/weightage.csv")
# pd.read_excel(open("data/01_BI_SQL_Dataset.xlsx", "rb"), sheet_name="partial_order_item_level").to_csv("data/01/partial_order_item_level.csv")
# pd.read_excel(open("data/01_BI_SQL_Dataset.xlsx", "rb"), sheet_name="item_category").to_csv("data/01/item_category.csv")

In [20]:
spark = SparkSession.builder\
                    .appName("sp-01-sql")\
                    .getOrCreate()

In [21]:
datasets = ["order_item", "first_purchase", "shop_level_target", "kam_level_target", "weightage", "partial_order_item_level", "item_category",]

In [22]:
for dataset in datasets:
    read_options = {
        "header": "true", 
        "inferSchema": "true",
    }

    spark.read.format("csv") \
                .options(**read_options) \
                .load(f"data/01/{dataset}.csv") \
                .createOrReplaceTempView(dataset)


In [23]:
spark.sql("SHOW TABLES").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|         |      first_purchase|       true|
|         |       item_category|       true|
|         |    kam_level_target|       true|
|         |          order_item|       true|
|         |partial_order_ite...|       true|
|         |   shop_level_target|       true|
|         |           weightage|       true|
+---------+--------------------+-----------+



In [24]:
query = """

WITH
TRANSACTIONS AS (
    SELECT
        buyer_id,
        DATE(create_datetime) AS dt,
        cluster_orders_item
    FROM 
        order_item
    WHERE
        DATE(create_datetime) BETWEEN "2021-04-01" AND "2021-04-30"
),

FIRST_TIME_BUY AS (
    SELECT
        buyer_id,
        DATE(result_first_purchase_time) AS dt
    FROM
        first_purchase
    WHERE
        DATE(result_first_purchase_time) BETWEEN "2021-04-01" AND "2021-04-30"
    
),

FILTERED_FIRST_TIME_BUY AS (
    SELECT
        FIRST_TIME_BUY.dt AS dt,
        FIRST_TIME_BUY.buyer_id AS buyer_id
    FROM
        FIRST_TIME_BUY
    LEFT JOIN
        TRANSACTIONS
        ON
            FIRST_TIME_BUY.buyer_id = TRANSACTIONS.buyer_id
    WHERE
        cluster_orders_item = "Fashion"
)

SELECT
    dt AS create_date,
    COUNT(*) AS num_of_first_purchase_buyers
FROM
   FILTERED_FIRST_TIME_BUY
GROUP BY
    dt
ORDER BY
    dt ASC
"""

In [25]:
spark.sql(query).show()

+-----------+----------------------------+
|create_date|num_of_first_purchase_buyers|
+-----------+----------------------------+
| 2021-04-13|                           1|
| 2021-04-15|                           2|
| 2021-04-25|                           1|
+-----------+----------------------------+



In [26]:
query_2 = """

WITH
TRANSACTIONS AS (
    SELECT
        buyer_id,
        DATE(create_datetime) AS dt,
        cluster_orders_item
    FROM 
        order_item
    WHERE
        DATE(create_datetime) BETWEEN "2021-04-01" AND "2021-04-30"
),

FIRST_TIME_BUY AS (
    SELECT
        buyer_id,
        DATE(result_first_purchase_time) AS dt
    FROM
        first_purchase
    WHERE
        DATE(result_first_purchase_time) BETWEEN "2021-04-01" AND "2021-04-30"
    
),

FILTERED_FIRST_TIME_BUY AS (
    SELECT
        FIRST_TIME_BUY.dt AS dt,
        FIRST_TIME_BUY.buyer_id AS buyer_id
    FROM
        FIRST_TIME_BUY
    LEFT JOIN
        TRANSACTIONS
        ON
            FIRST_TIME_BUY.buyer_id = TRANSACTIONS.buyer_id
    WHERE
        cluster_orders_item = "Fashion"
),

FINAL_FIRST_TIME_BUY AS (
    SELECT
        dt,
        COUNT(*) AS num_of_first_purchase_buyers
    FROM
        FILTERED_FIRST_TIME_BUY
    GROUP BY
        dt
),

DATE_RANGE AS (
    SELECT
        EXPLODE(SEQUENCE(TO_DATE('2021-04-01'), 
        TO_DATE('2021-04-30'), INTERVAL 1 DAY)) AS create_date
)

SELECT
    create_date,
    COALESCE(num_of_first_purchase_buyers, 0) AS num_of_first_purchase_buyers
FROM
    DATE_RANGE
LEFT JOIN
    FINAL_FIRST_TIME_BUY
    ON
    DATE_RANGE.create_date = FINAL_FIRST_TIME_BUY.dt
ORDER BY
    create_date ASC
"""

In [27]:
spark.sql(query_2).show(30)

+-----------+----------------------------+
|create_date|num_of_first_purchase_buyers|
+-----------+----------------------------+
| 2021-04-01|                           0|
| 2021-04-02|                           0|
| 2021-04-03|                           0|
| 2021-04-04|                           0|
| 2021-04-05|                           0|
| 2021-04-06|                           0|
| 2021-04-07|                           0|
| 2021-04-08|                           0|
| 2021-04-09|                           0|
| 2021-04-10|                           0|
| 2021-04-11|                           0|
| 2021-04-12|                           0|
| 2021-04-13|                           1|
| 2021-04-14|                           0|
| 2021-04-15|                           2|
| 2021-04-16|                           0|
| 2021-04-17|                           0|
| 2021-04-18|                           0|
| 2021-04-19|                           0|
| 2021-04-20|                           0|
| 2021-04-2

# Q1.3

In [28]:
spark.sql("SHOW TABLES").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|         |      first_purchase|       true|
|         |       item_category|       true|
|         |    kam_level_target|       true|
|         |          order_item|       true|
|         |partial_order_ite...|       true|
|         |   shop_level_target|       true|
|         |           weightage|       true|
+---------+--------------------+-----------+



In [29]:
query = """
WITH
TOTAL_ORDER AS (
    SELECT 
        partial_order_item_level.order_id,
        partial_order_item_level.item_id,
        DATE(partial_order_item_level.create_timestamp) AS create_date,
        item_category.cluster
    FROM
        partial_order_item_level
    LEFT JOIN
        item_category
        ON
            partial_order_item_level.item_id = item_category.item_id
    WHERE
        DATE(partial_order_item_level.create_timestamp) BETWEEN "2024-07-01" AND "2024-07-31"
),

CLUSTER_ITEM_COUNT AS (
    SELECT
        cluster,
        item_id,
        COUNT(DISTINCT order_id) AS total_item_per_cluster
    FROM
        TOTAL_ORDER
    GROUP BY
        cluster, item_id
),

CLUSTER_ORDER_COUNT AS (
    SELECT
        cluster,
        COUNT(DISTINCT order_id) AS total_order_per_cluster
    FROM
        TOTAL_ORDER
    GROUP BY
        cluster
),

CALCULATE_CONTRIBUTION AS (
    SELECT
        CLUSTER_ITEM_COUNT.cluster,
        item_id,
        CLUSTER_ITEM_COUNT.total_item_per_cluster,
        CLUSTER_ORDER_COUNT.total_order_per_cluster,
        CLUSTER_ITEM_COUNT.total_item_per_cluster / CLUSTER_ORDER_COUNT.total_order_per_cluster AS contribution_rate
    FROM
        CLUSTER_ITEM_COUNT
    LEFT JOIN
        CLUSTER_ORDER_COUNT
        ON
            CLUSTER_ITEM_COUNT.cluster = CLUSTER_ORDER_COUNT.cluster 
),

RANK_CONTRIBUTION AS (
    SELECT
        *,
        RANK() OVER (PARTITION BY cluster ORDER BY contribution_rate DESC) AS contribution_rank
    FROM
        CALCULATE_CONTRIBUTION
)

SELECT
    item_id,
    cluster
FROM
    RANK_CONTRIBUTION
WHERE
    contribution_rate > 0.5
ORDER BY
    cluster, contribution_rank DESC
"""
spark.sql(query).show(50)

+-------+-------+
|item_id|cluster|
+-------+-------+
+-------+-------+



In [30]:
query = """
SELECT *
FROM
    partial_order_item_level
"""
spark.sql(query).show()

+---+-------------------+----------+---------+---------------+-----------+
|_c0|   create_timestamp|  buyer_id|  shop_id|       order_id|    item_id|
+---+-------------------+----------+---------+---------------+-----------+
|  0|2024-07-01 00:00:01| 239569179|159375169|173462401278157|24373608531|
|  1|2024-07-01 00:00:04| 119328142|  2947308|173462403232581|17999124990|
|  2|2024-07-01 00:00:10|1278919050|139118063|173462410268531|17474241988|
|  3|2024-07-01 00:00:26|1048757650|165561030|173462425282927| 3965607329|
|  4|2024-07-01 00:00:30| 566548738| 43774111|173462430210862|18169086361|
|  5|2024-07-01 00:00:49| 136339849|420466609|173462448286550|10099535909|
|  6|2024-07-01 00:00:56|  82665348| 36641085|173462456289445|13172996330|
|  7|2024-07-01 00:01:06| 143232789|  9940980|173462465261721|25382845612|
|  8|2024-07-01 00:01:08| 786000874| 80618511|173462468217410|23976199827|
|  9|2024-07-01 00:01:16| 146081114| 64535236|173462476213778| 6585327147|
| 10|2024-07-01 00:01:43|

In [31]:
query = """
SELECT 
    COUNT(DISTINCT order_id)
FROM
    partial_order_item_level
"""
spark.sql(query).show()

+------------------------+
|count(DISTINCT order_id)|
+------------------------+
|                      73|
+------------------------+



In [32]:
query = """
SELECT 
    COUNT(DISTINCT order_id, item_id)
FROM
    partial_order_item_level
"""
spark.sql(query).show()

+---------------------------------+
|count(DISTINCT order_id, item_id)|
+---------------------------------+
|                               89|
+---------------------------------+



In [33]:
query = """
SELECT 
    COUNT(DISTINCT item_id)
FROM
    partial_order_item_level
"""
spark.sql(query).show()

+-----------------------+
|count(DISTINCT item_id)|
+-----------------------+
|                     89|
+-----------------------+



In [34]:
query = """
SELECT
    order_id,
    item_id,
    COUNT(*)
FROM
    partial_order_item_level
GROUP BY 1,2
ORDER BY item_id
"""
spark.sql(query).show()

+---------------+----------+--------+
|       order_id|   item_id|count(1)|
+---------------+----------+--------+
|173463513220909|1237830184|       6|
|173462593220301|2296384316|       4|
|173463036282645|2378278433|       2|
|173462685267365|3452298053|       2|
|173462425282927|3965607329|       2|
|173463673246420|4097708038|       2|
|173463366261131|4735972976|       2|
|173463409201770|5538292301|       2|
|173462581281750|6021435898|       2|
|173462476213778|6585327147|       2|
|173463355206866|6730658842|       2|
|173463907232386|6952476724|       2|
|173463355206866|7230672083|       2|
|173463106268213|7297953620|       2|
|173463678266182|7497452982|       2|
|173463907232386|7996634182|       2|
|173463938280863|8980834747|       2|
|173464243283288|8994199288|       2|
|173463378216814|9927258415|       2|
|173464045295892|9944277794|       2|
+---------------+----------+--------+
only showing top 20 rows



In [35]:
query = """
SELECT 
    *
FROM
    partial_order_item_level
WHERE
    order_id = 173463513220909
"""
spark.sql(query).show()

+---+-------------------+---------+--------+---------------+----------+
|_c0|   create_timestamp| buyer_id| shop_id|       order_id|   item_id|
+---+-------------------+---------+--------+---------------+----------+
| 64|2024-07-01 00:18:34|158034339|73867041|173463513220909|1237830184|
| 65|2024-07-01 00:18:34|158034339|73867041|173463513220909|1237830184|
| 66|2024-07-01 00:18:34|158034339|73867041|173463513220909|1237830184|
|164|2024-06-01 00:18:34|158034339|73867041|173463513220909|1237830184|
|165|2024-06-01 00:18:34|158034339|73867041|173463513220909|1237830184|
|166|2024-06-01 00:18:34|158034339|73867041|173463513220909|1237830184|
+---+-------------------+---------+--------+---------------+----------+



In [36]:
spark.stop()

# QA3: Pyspark

In [37]:
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import *
from pyspark.sql.functions import format_number
# import pandas as pd

In [38]:
# pd.read_excel(open("data/03 Python test and Dataset.xlsx", "rb"), sheet_name="pricing_project_dataset").to_csv("data/pricing_project_dataset.csv")
# pd.read_excel(open("data/03 Python test and Dataset.xlsx", "rb"), sheet_name="platform_number").to_csv("data/platform_number.csv")
# pd.read_excel(open("data/03 Python test and Dataset.xlsx", "rb"), sheet_name="exchange_rate").to_csv("data/exchange_rate.csv")

In [39]:
spark = SparkSession.builder \
                    .appName("sp-03-spark") \
                    .master("local[8]") \
                    .getOrCreate()

# backup code of using jars method
# .config("spark.jars", "src/spark-excel_2.13-3.5.1_0.20.4.jar") \
# .config("spark.jars.packages", "com.crealytics:spark-excel_3.5.1:0.20.4") \

In [40]:
read_options = {
    "header": "true", 
    "inferSchema": "true",
}

data_pricing_path = "data/pricing_project_dataset.csv"
data_platform_number_path = "data/platform_number.csv"

df_pricing = spark.read.format("csv") \
                        .options(**read_options) \
                        .load(data_pricing_path)

df_platform_order = spark.read.format("csv") \
                        .options(**read_options) \
                        .load(data_platform_number_path)

In [41]:
df_platform_order.show(5)

+---+------+--------------+----------------+
|_c0|region|platform order|platform_gmv_usd|
+---+------+--------------+----------------+
|  0|    ID|    1553364.13|  2.4388379495E8|
|  1|    MY|     333184.27|   7.341683753E7|
|  2|    PH|     392687.73|   7.538330311E7|
|  3|    SG|     526181.77|   9.917029894E7|
|  4|    TH|    1018358.54|  1.7340089962E8|
+---+------+--------------+----------------+
only showing top 5 rows



In [42]:
df_pricing.show(5)

+---+------------+--------------+--------------+--------------+---------------+------------------+-------------------+----------------------+------------------+----------------------+------------------+------------+-----------------------------------+
|_c0|grass_region|category_group|   seller_type|shopee_item_id|shopee_model_id|competitor_item_id|competitor_model_id|shopee_model_price_usd|shopee_model_price|competitor_model_price|    shopee_gmv_usd|shopee_order|shopee_model_competitiveness_status|
+---+------------+--------------+--------------+--------------+---------------+------------------+-------------------+----------------------+------------------+----------------------+------------------+------------+-----------------------------------+
|  0|          ID|   Electronics|Long Tail (LT)|    5646734211|     3953474697|        9153508336|          992466002|     50.33644918112619|            779233|                647903|  8758.53764413294|         174|                       Shopee

In [43]:
clean_df_platform = df_platform_order.withColumnRenamed("platform order", "platform_order")

In [44]:
clean_df_pricing = df_pricing.withColumnRenamed("grass_region", "region")

In [45]:
clean_df_platform.show()

+---+------+--------------+----------------+
|_c0|region|platform_order|platform_gmv_usd|
+---+------+--------------+----------------+
|  0|    ID|    1553364.13|  2.4388379495E8|
|  1|    MY|     333184.27|   7.341683753E7|
|  2|    PH|     392687.73|   7.538330311E7|
|  3|    SG|     526181.77|   9.917029894E7|
|  4|    TH|    1018358.54|  1.7340089962E8|
|  5|    VN|     413006.15|   7.454916352E7|
+---+------+--------------+----------------+



In [46]:
clean_df_pricing.show()

+---+------+--------------+--------------+--------------+---------------+------------------+-------------------+----------------------+------------------+----------------------+------------------+------------+-----------------------------------+
|_c0|region|category_group|   seller_type|shopee_item_id|shopee_model_id|competitor_item_id|competitor_model_id|shopee_model_price_usd|shopee_model_price|competitor_model_price|    shopee_gmv_usd|shopee_order|shopee_model_competitiveness_status|
+---+------+--------------+--------------+--------------+---------------+------------------+-------------------+----------------------+------------------+----------------------+------------------+------------+-----------------------------------+
|  0|    ID|   Electronics|Long Tail (LT)|    5646734211|     3953474697|        9153508336|          992466002|     50.33644918112619|            779233|                647903|  8758.53764413294|         174|                       Shopee > CPT|
|  1|    SG|    

## Q3.1

In [47]:
agg_df_pricing = clean_df_pricing.groupBy(["region"]) \
                                    .agg(sum("shopee_order").alias("sum_shopee_order"))

In [48]:
# clean_df_pricing = clean_df_pricing.groupBy(["grass_region", "shopee_item_id"]) \
#                                     .agg(
#                                         sum("shopee_order").alias("sum_shopee_order"),
#                                         # count("shopee_item_id").alias("number_of_item")
#                                     ) \
#                                     .withColumnRenamed("grass_region", "region")
#                                     # .sort("number_of_item", ascending=False)

In [49]:
agg_df_pricing.show(5)

+------+----------------+
|region|sum_shopee_order|
+------+----------------+
|    ID|          848601|
|    MY|          207739|
|    SG|          219624|
|    PH|          202688|
|    TH|          408024|
+------+----------------+
only showing top 5 rows



In [50]:
clean_df_platform.show()

+---+------+--------------+----------------+
|_c0|region|platform_order|platform_gmv_usd|
+---+------+--------------+----------------+
|  0|    ID|    1553364.13|  2.4388379495E8|
|  1|    MY|     333184.27|   7.341683753E7|
|  2|    PH|     392687.73|   7.538330311E7|
|  3|    SG|     526181.77|   9.917029894E7|
|  4|    TH|    1018358.54|  1.7340089962E8|
|  5|    VN|     413006.15|   7.454916352E7|
+---+------+--------------+----------------+



In [51]:
agg_df_pricing = agg_df_pricing.join(clean_df_platform, on="region", how="left")\
                                .withColumn("order_coverage_by_region", format_number(col("sum_shopee_order")/col("platform_order"), 6)) \
                                .select("region", "order_coverage_by_region")

In [52]:
agg_df_pricing.show()

+------+------------------------+
|region|order_coverage_by_region|
+------+------------------------+
|    ID|                0.546299|
|    MY|                0.623496|
|    SG|                0.417392|
|    PH|                0.516156|
|    TH|                0.400668|
|    VN|                0.516903|
+------+------------------------+



In [53]:
number_item_pricing = clean_df_pricing.groupBy(["region"]) \
                                    .agg(count("shopee_item_id").alias("number_of_item_by_region"))

In [54]:
number_item_pricing.show()

+------+------------------------+
|region|number_of_item_by_region|
+------+------------------------+
|    ID|                    8428|
|    MY|                    2118|
|    SG|                    2194|
|    PH|                    2025|
|    TH|                    4137|
|    VN|                    2097|
+------+------------------------+



In [55]:
agg_df_pricing = agg_df_pricing.join(number_item_pricing, on="region", how="left")

In [56]:
# df_pricing.registerTempTable("local_price_competitive_by_sku")

In [57]:
clean_df_pricing.show()

+---+------+--------------+--------------+--------------+---------------+------------------+-------------------+----------------------+------------------+----------------------+------------------+------------+-----------------------------------+
|_c0|region|category_group|   seller_type|shopee_item_id|shopee_model_id|competitor_item_id|competitor_model_id|shopee_model_price_usd|shopee_model_price|competitor_model_price|    shopee_gmv_usd|shopee_order|shopee_model_competitiveness_status|
+---+------+--------------+--------------+--------------+---------------+------------------+-------------------+----------------------+------------------+----------------------+------------------+------------+-----------------------------------+
|  0|    ID|   Electronics|Long Tail (LT)|    5646734211|     3953474697|        9153508336|          992466002|     50.33644918112619|            779233|                647903|  8758.53764413294|         174|                       Shopee > CPT|
|  1|    SG|    

In [58]:
intermediate_net_comp = clean_df_pricing.groupBy(["region", "shopee_model_competitiveness_status"]) \
                                        .agg(count("shopee_model_id").alias("number_of_shopee_model_id"))

In [59]:
intermediate_net_comp.show()

+------+-----------------------------------+-------------------------+
|region|shopee_model_competitiveness_status|number_of_shopee_model_id|
+------+-----------------------------------+-------------------------+
|    VN|                       Shopee < CPT|                     1035|
|    PH|                       Shopee > CPT|                     1006|
|    PH|                       Shopee < CPT|                     1011|
|    TH|                       Shopee > CPT|                     2045|
|    SG|                       Shopee > CPT|                     1085|
|    VN|                       Shopee > CPT|                     1062|
|    TH|                       Shopee = CPT|                       23|
|    SG|                       Shopee = CPT|                      159|
|    SG|                       Shopee < CPT|                      950|
|    PH|                       Shopee = CPT|                        8|
|    ID|                       Shopee < CPT|                     4222|
|    M

In [60]:
win_comp = intermediate_net_comp.where(col("shopee_model_competitiveness_status") == "Shopee > CPT").withColumnRenamed("number_of_shopee_model_id", "number_of_larger_shopee_model")
lose_comp = intermediate_net_comp.where(col("shopee_model_competitiveness_status") == "Shopee < CPT").withColumnRenamed("number_of_shopee_model_id", "number_of_lower_shopee_model")
total_comp = clean_df_pricing.groupBy(["region"]).agg(count("shopee_model_id").alias("total_shopee_model"))

In [61]:
final_net_comp = win_comp.join(lose_comp, on="region", how="left")\
                            .join(total_comp, on="region", how="left")\
                            .select("region", "number_of_larger_shopee_model", "number_of_lower_shopee_model", "total_shopee_model")\
                            .withColumn("net_competitiveness", format_number((col("number_of_lower_shopee_model") - col("number_of_larger_shopee_model")) / col("total_shopee_model"), 6))\
                            .select("region", "net_competitiveness")

In [62]:
final_net_comp.show()

+------+-------------------+
|region|net_competitiveness|
+------+-------------------+
|    PH|           0.002469|
|    TH|           0.005801|
|    SG|          -0.061531|
|    VN|          -0.012876|
|    MY|           0.002833|
|    ID|           0.001898|
+------+-------------------+



In [63]:
final_net_comp = final_net_comp.join(agg_df_pricing, on="region", how="left")

In [64]:
final_net_comp.select("region", "order_coverage_by_region", "net_competitiveness", "number_of_item_by_region").show()

+------+------------------------+-------------------+------------------------+
|region|order_coverage_by_region|net_competitiveness|number_of_item_by_region|
+------+------------------------+-------------------+------------------------+
|    PH|                0.516156|           0.002469|                    2025|
|    TH|                0.400668|           0.005801|                    4137|
|    SG|                0.417392|          -0.061531|                    2194|
|    VN|                0.516903|          -0.012876|                    2097|
|    MY|                0.623496|           0.002833|                    2118|
|    ID|                0.546299|           0.001898|                    8428|
+------+------------------------+-------------------+------------------------+



## Q3.2

In [65]:
clean_df_pricing.show()

+---+------+--------------+--------------+--------------+---------------+------------------+-------------------+----------------------+------------------+----------------------+------------------+------------+-----------------------------------+
|_c0|region|category_group|   seller_type|shopee_item_id|shopee_model_id|competitor_item_id|competitor_model_id|shopee_model_price_usd|shopee_model_price|competitor_model_price|    shopee_gmv_usd|shopee_order|shopee_model_competitiveness_status|
+---+------+--------------+--------------+--------------+---------------+------------------+-------------------+----------------------+------------------+----------------------+------------------+------------+-----------------------------------+
|  0|    ID|   Electronics|Long Tail (LT)|    5646734211|     3953474697|        9153508336|          992466002|     50.33644918112619|            779233|                647903|  8758.53764413294|         174|                       Shopee > CPT|
|  1|    SG|    

In [66]:
clean_df_pricing.select("shopee_item_id", "shopee_model_id", "shopee_model_competitiveness_status") \
                .withColumn("new_cpt_status", 
                    when(col("shopee_model_competitiveness_status") == "Shopee < CPT", 1) \
                    .when(col("shopee_model_competitiveness_status") == "Shopee = CPT", 2) \
                    .when(col("shopee_model_competitiveness_status") == "Shopee > CPT", 3) \
                    .otherwise(4)
                )\
                .select(
                    "shopee_item_id",
                    "shopee_model_id",
                    "shopee_model_competitiveness_status",
                    "new_cpt_status",
                    min("new_cpt_status").over(Window.partitionBy(["shopee_item_id", "shopee_model_id"])).alias("priority")
                )\
                .where(col("new_cpt_status") == col("priority"))\
                .select(
                    "shopee_item_id",
                    "shopee_model_id",
                    "shopee_model_competitiveness_status"
                ) \
                .show()

+--------------+---------------+-----------------------------------+
|shopee_item_id|shopee_model_id|shopee_model_competitiveness_status|
+--------------+---------------+-----------------------------------+
|        580861|     9019764370|                       Shopee > CPT|
|       1323167|     6397942095|                       Shopee < CPT|
|       1837669|     8342684624|                       Shopee < CPT|
|       2650112|     7803527018|                       Shopee < CPT|
|       2666411|     3127858767|                       Shopee < CPT|
|       2794532|     8859393605|                       Shopee < CPT|
|       2996790|     7322469724|                       Shopee > CPT|
|       3202936|     8829997338|                       Shopee < CPT|
|       3239333|      747032931|                       Shopee > CPT|
|       3375995|     5926043907|                       Shopee > CPT|
|       3458546|     6882920226|                       Shopee < CPT|
|       3574775|     7582427211|  

## Q3.3

In [67]:
calculate_order = clean_df_pricing.select("region", "shopee_item_id", "shopee_model_id", "shopee_order")

In [68]:
calculate_order.show(2)

+------+--------------+---------------+------------+
|region|shopee_item_id|shopee_model_id|shopee_order|
+------+--------------+---------------+------------+
|    ID|    5646734211|     3953474697|         174|
|    SG|    3735125711|     6296885988|         187|
+------+--------------+---------------+------------+
only showing top 2 rows



In [69]:
calculate_platform = clean_df_platform.select("region", "platform_order")

In [70]:
calculate_platform.show(2)

+------+--------------+
|region|platform_order|
+------+--------------+
|    ID|    1553364.13|
|    MY|     333184.27|
+------+--------------+
only showing top 2 rows



In [71]:
final_order = calculate_order.groupBy(["region", "shopee_item_id", "shopee_model_id"])\
                            .agg(sum("shopee_order").alias("sum_shopee_order")) \
                            .join(calculate_platform, on="region", how="left")\
                            .withColumn("contribute_ratio", format_number(col("sum_shopee_order") / col("platform_order"), 10)) \
                            .withColumn("order_threshold", format_number(col("sum_shopee_order")*0.3, 0)) \
                            .withColumn("contribution_rank", row_number().over(Window.partitionBy(["region", "shopee_item_id", "shopee_model_id"]).orderBy(col("contribute_ratio").desc()))) \
                            .where(col("contribution_rank") <= col("order_threshold"))

In [72]:
final_order.groupBy(["region"])\
            .agg(
                count_distinct("shopee_item_id").alias("number_of_item_tops_30")
            ) \
            .show()

+------+----------------------+
|region|number_of_item_tops_30|
+------+----------------------+
|    ID|                  8382|
|    MY|                  2111|
|    SG|                  2186|
|    PH|                  2012|
|    TH|                  4111|
|    VN|                  2088|
+------+----------------------+



## Q3.5

In [73]:
clean_df_pricing.count()

20999

In [74]:
final_order.count()

20890

In [75]:
final_order.show()

+------+--------------+---------------+----------------+--------------+----------------+---------------+-----------------+
|region|shopee_item_id|shopee_model_id|sum_shopee_order|platform_order|contribute_ratio|order_threshold|contribution_rank|
+------+--------------+---------------+----------------+--------------+----------------+---------------+-----------------+
|    ID|        580861|     9019764370|             178|    1553364.13|    0.0001145900|             53|                1|
|    ID|       1837669|     8342684624|              42|    1553364.13|    0.0000270381|             13|                1|
|    ID|       2666411|     3127858767|             137|    1553364.13|    0.0000881957|             41|                1|
|    ID|       3375995|     5926043907|              32|    1553364.13|    0.0000206004|             10|                1|
|    ID|       3458546|     6882920226|              38|    1553364.13|    0.0000244630|             11|                1|
|    ID|       4

In [76]:
top_30_model = final_order.select("region", "shopee_item_id", "shopee_model_id")\
                            .withColumn("is_top_30_model", lit(1))
clean_df_pricing.join(top_30_model, on=["region", "shopee_item_id", "shopee_model_id"], how="left") \
                .fillna({"is_top_30_model": 0}) \
                .drop("_c0") \
                .createOrReplaceTempView("local_price_competitive_by_sku")

                # .where(col("is_top_30_model").isNull()) \

In [77]:
spark.stop()